## 第7章 数据清洗和准备

在本章中，我会讨论处理缺失数据、重复数据、字符串操作和其
它分析数据转换的⼯具。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



/home/ubuntu/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### 7.1 处理缺失数据


对于数值数据， pandas使⽤浮点值
NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以
⽅便的检测出来：

In [5]:
string_data = pd.Series(['asdfsdf', 'wtrafda', np.nan, 'asdfew'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

将缺失值表示为
NA，它表示不可⽤not available。在统计应⽤中， NA数据可能
是不存在的数据或者虽然存在，但是没有观察到（例如，数据采
集中发⽣了问题）。当进⾏数据清洗以进⾏分析时，最好直接对
缺失数据进⾏分析，以判断数据采集的问题或缺失数据可能导致
的偏差。

Python内置的None值在对象数组中也可以作为NA：

In [7]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 滤除缺失数据

过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布
尔索引的⼿⼯⽅法，但dropna可能会更实⽤⼀些。对于⼀个
Series， dropna返回⼀个仅含⾮空数据和索引值的Series：

In [8]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

这等价于

In [9]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

⽽对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全
NA或含有NA的⾏或列。 dropna默认丢弃任何含有缺失值的⾏：

In [12]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 3.2, 12.]])
cleaned = data.dropna()
print(data)
print(cleaned)

     0    1     2
0  1.0  6.5   3.0
1  1.0  NaN   NaN
2  NaN  NaN   NaN
3  NaN  3.2  12.0
     0    1    2
0  1.0  6.5  3.0


传⼊how='all'将只丢弃全为NA的那些⾏

In [14]:
data.dropna(how='all')  # 全为NA的行被丢弃

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,3.2,12.0


In [17]:
data[4] = NA
print(data)
print()
data.dropna(axis=1, how='all')

     0    1     2   4
0  1.0  6.5   3.0 NaN
1  1.0  NaN   NaN NaN
2  NaN  NaN   NaN NaN
3  NaN  3.2  12.0 NaN



,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,3.2,12.0


另⼀个滤除DataFrame⾏的问题涉及时间序列数据。假设你只想
留下⼀部分观测数据，可以⽤thresh参数实现此⽬的

**其实thresh=n不是滤除n行，而且保留至少有n个非NaN数据的行**

In [44]:
np.random.seed(2018)

df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df.iloc[4:6, 0] = NA
print(df)
print()
print(df.dropna())
# print(df.dropna(thresh=2))
print()
print(df.dropna(thresh=1))
print()
print(df.dropna(thresh=2))
print()
print()
print(df.dropna(thresh=3))

          0         1         2
0 -0.276768       NaN       NaN
1 -1.279487       NaN       NaN
2 -0.142790       NaN -0.688065
3  0.433564       NaN -0.165131
4       NaN  0.546631  1.230655
5       NaN -1.210625 -0.306677
6 -1.057419  0.402057  0.289165

          0         1         2
6 -1.057419  0.402057  0.289165

          0         1         2
0 -0.276768       NaN       NaN
1 -1.279487       NaN       NaN
2 -0.142790       NaN -0.688065
3  0.433564       NaN -0.165131
4       NaN  0.546631  1.230655
5       NaN -1.210625 -0.306677
6 -1.057419  0.402057  0.289165

          0         1         2
2 -0.142790       NaN -0.688065
3  0.433564       NaN -0.165131
4       NaN  0.546631  1.230655
5       NaN -1.210625 -0.306677
6 -1.057419  0.402057  0.289165


          0         1         2
6 -1.057419  0.402057  0.289165


### 填充缺失数据
你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数
据），⽽是希望通过其他⽅式填补那些“空洞”。对于⼤多数情况
⽽⾔，

fillna⽅法是最主要的函数。通过⼀个常数调⽤fillna就会将
缺失值替换为那个常数值

In [45]:
df.fillna(0)

,0,1,2
0,-0.276768,0.000000,0.000000
1,-1.279487,0.000000,0.000000
2,-0.142790,0.000000,-0.688065
3,0.433564,0.000000,-0.165131
4,0.000000,0.546631,1.230655
5,0.000000,-1.210625,-0.306677
6,-1.057419,0.402057,0.289165


若是通过⼀个字典调⽤fillna，就可以实现对不同的列填充不同的
值：

In [46]:
df.fillna({1:0.5, 2:8.8})

,0,1,2
0,-0.276768,0.500000,8.800000
1,-1.279487,0.500000,8.800000
2,-0.142790,0.500000,-0.688065
3,0.433564,0.500000,-0.165131
4,NaN,0.546631,1.230655
5,NaN,-1.210625,-0.306677
6,-1.057419,0.402057,0.289165


fillna默认会返回新对象，但也可以对现有对象进⾏就地修改：

In [48]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.276768,0.000000,0.000000
1,-1.279487,0.000000,0.000000
2,-0.142790,0.000000,-0.688065
3,0.433564,0.000000,-0.165131
4,0.000000,0.546631,1.230655
5,0.000000,-1.210625,-0.306677
6,-1.057419,0.402057,0.289165


对reindexing有效的那些插值⽅法也可⽤于fillna：

In [49]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,1.282733,-1.065696,-1.706633
1,-0.172797,0.063710,0.370628
2,-1.604543,NaN,0.380370
3,-0.276501,NaN,0.421854
4,0.704780,NaN,NaN
5,0.138125,NaN,NaN


In [53]:
# method : {‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None}, default None
# pad/ffill：用前一个非缺失值去填充该缺失值
# backfill/bfill：用下一个非缺失值填充该缺失值
df.fillna(method='ffill')

,0,1,2
0,1.282733,-1.065696,-1.706633
1,-0.172797,0.063710,0.370628
2,-1.604543,0.063710,0.380370
3,-0.276501,0.063710,0.421854
4,0.704780,0.063710,0.421854
5,0.138125,0.063710,0.421854


In [54]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,1.282733,-1.065696,-1.706633
1,-0.172797,0.063710,0.370628
2,-1.604543,0.063710,0.380370
3,-0.276501,0.063710,0.421854
4,0.704780,NaN,0.421854
5,0.138125,NaN,0.421854


In [56]:
# 还可以插入Series的平均值或中位数
data = pd.Series([1, NA, 3.5, 6.2, NA, 8])

data.fillna(data.mean())

0    1.000
1    4.675
2    3.500
3    6.200
4    4.675
5    8.000
dtype: float64

### 7.2数据转换

### 移除重复数据

In [57]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的duplicated⽅法返回⼀个布尔型Series，表示各⾏
是否是重复⾏（前⾯出现过的⾏）

In [58]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有⼀个与此相关的drop_duplicates⽅法，它会返回⼀个
DataFrame，重复的数组会标为False

In [59]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个⽅法默认会判断全部列，你也可以指定部分列进⾏重复项
判断。假设我们还有⼀列值，且只希望根据k1列过滤重复项：

In [62]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [63]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicates默认保留的是第⼀个出现的值组
合。传⼊keep='last'则保留最后⼀个

In [64]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 利用函数映射进行数据转换

中的值来实现转换⼯作。我们来看看下⾯这组有关⾁类的数据：

In [66]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想要添加⼀列表示该⾁类⻝物来源的动物类型。我们先编
写⼀个不同⾁类到动物的映射：

In [68]:

meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Series的map⽅法可以接受⼀个函数或含有映射关系的字典型对
象，但是这⾥有⼀个⼩问题，即有些⾁类的⾸字⺟⼤写了，⽽另
⼀些则没有。因此，我们还需要使⽤Series的str.lower⽅法，将
各个值转换为⼩写：

In [69]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [70]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以传⼊⼀个能够完成全部这些⼯作的函数：

In [74]:
data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon
